# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
df_q1 = df[df["Gold"] == df["Gold"].max()]

In [4]:
def answer_one():
    return df_q1.index.values[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
df_q2 = df.copy()
df_q2["Gold_diff"] = (df["Gold"] - df["Gold.1"])

In [6]:
def answer_two():
    return df_q2["Gold_diff"].idxmax()

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [7]:
df_q3 = df[(df["Gold"] > 0) & (df["Gold.1"] > 0)]
df_q3 = (df_q3["Gold"] - df_q3["Gold.1"]) / df_q3["Gold.2"]

In [8]:
def answer_three():
    return df_q3.idxmax()

answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [9]:
df_q4 = df.copy()
df_q4["Points"] = df_q4["Gold.2"] * 3 + df_q4["Silver.2"] * 2 + df_q4["Bronze.2"] * 1

In [10]:
def answer_four():
    return df_q4.Points

answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

In [11]:
census_df = pd.read_csv('census.csv')

In [12]:
census_df.isna().values.any() # keine NaN Werte in census_df

False

In [13]:
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [14]:
census_df_states = census_df[(census_df["SUMLEV"] == 40)]
census_df_states.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
68,40,4,9,2,0,Alaska,Alaska,710231,710249,714021,...,-1.173489,-1.946424,-3.915107,-14.438910,-10.407475,0.931274,1.818497,-0.757148,-11.271709,-6.881838
98,40,4,8,4,0,Arizona,Arizona,6392017,6392307,6408208,...,1.327489,5.245740,3.905473,6.219955,6.776501,3.290378,7.337279,6.123606,8.761352,9.335208
114,40,3,7,5,0,Arkansas,Arkansas,2915918,2915958,2922394,...,1.365312,-0.432402,-0.442153,-1.060966,-0.407735,2.326251,0.646395,0.682189,0.226168,0.901928
190,40,4,9,6,0,California,California,37253956,37254503,37334079,...,-1.148464,-1.163788,-1.339869,-0.862856,-1.981572,2.761704,2.647127,2.728645,3.743342,2.656065


In [15]:
census_df_counties = census_df[(census_df["SUMLEV"] == 50)]
census_df_counties.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [16]:
census_df_counties_grouped_stname = census_df_counties.groupby("STNAME").count()

In [17]:
def answer_five():
    return census_df_counties_grouped_stname["COUNTY"].idxmax()

answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [18]:
census_df_states_sorted_pop2010 = census_df_counties[["STNAME", "CENSUS2010POP"]].sort_values(["STNAME", "CENSUS2010POP"], ascending = [True , False])
census_df_states_sorted_pop2010_top3 = census_df_states_sorted_pop2010.groupby("STNAME").head(3)
census_df_states_sorted_pop2010_top3_sum = census_df_states_sorted_pop2010_top3.groupby("STNAME").sum()

In [19]:
def answer_six():
    return list(census_df_states_sorted_pop2010_top3_sum.nlargest(3, "CENSUS2010POP").index.values)

answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [20]:
import numpy as np

In [21]:
census_df_counties_popest = census_df_counties[['CTYNAME','POPESTIMATE2010', 'POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']]

In [22]:
# map? map(func, *iterables) --> map object
# Für jedes County den Maximalwert der Popesimate Spalten finden
census_df_counties_popest_max = map(max,
                                    census_df_counties_popest['POPESTIMATE2010'],
                                    census_df_counties_popest['POPESTIMATE2011'],
                                    census_df_counties_popest['POPESTIMATE2012'],
                                    census_df_counties_popest['POPESTIMATE2013'],
                                    census_df_counties_popest['POPESTIMATE2014'],
                                    census_df_counties_popest['POPESTIMATE2015'],
                                    )
census_df_counties_popest_max_array = np.array(list(census_df_counties_popest_max))

In [23]:
# Für jedes County den Minimalwert der Popesimate Spalten finden
census_df_counties_popest_min = map(min,
                                    census_df_counties_popest['POPESTIMATE2010'],
                                    census_df_counties_popest['POPESTIMATE2011'],
                                    census_df_counties_popest['POPESTIMATE2012'],
                                    census_df_counties_popest['POPESTIMATE2013'],
                                    census_df_counties_popest['POPESTIMATE2014'],
                                    census_df_counties_popest['POPESTIMATE2015'],
                                    )
census_df_counties_popest_min_array = np.array(list(census_df_counties_popest_min))

In [24]:
# Differenz zwischen max und min Array berechnen
census_df_counties_popest_diff = census_df_counties_popest_max_array - census_df_counties_popest_min_array

In [25]:
# argmax() Return int position of the largest value in the Series.
census_df_counties_popest.iloc[census_df_counties_popest_diff.argmax()]

CTYNAME            Harris County
POPESTIMATE2010          4108187
POPESTIMATE2011          4181238
POPESTIMATE2012          4262504
POPESTIMATE2013          4352462
POPESTIMATE2014          4447577
POPESTIMATE2015          4538028
Name: 2667, dtype: object

In [26]:
def answer_seven():
    return census_df_counties_popest.iloc[census_df_counties_popest_diff.argmax()].values[0]

answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [27]:
census_df_query = census_df[(census_df["SUMLEV"] == 50) &
                            (census_df["REGION"] <= 2) &
                            (census_df["CTYNAME"].str.startswith("Washington")) &
                            (census_df["POPESTIMATE2015"] > census_df['POPESTIMATE2014'])]

In [28]:
def answer_eight():
    return census_df_query[["STNAME", "CTYNAME"]].sort_index()

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
